# Building an LLM App for Document Retrieval / Extraction
This tutorial shows how to build a basic LLM App for retrival-augmented question-answering.
* **Dataset:** YouTube transcripts
* **Feature store:** chromadb
* Track datasets and embeddings as artifacts
* Track prompts and chain executions
* Log token counts and cost

## 1. Setup

In [ ]:
!pip install -qqq wandb langchain pytube tiktoken openai youtube-transcript-api chromadb langchain-openai gradio

In [ ]:
# TODO: remove this cell
import os

# set openApi key in os variables

In [ ]:
from getpass import getpass
import os

if os.getenv("OPENAI_API_KEY") is None:
  if any(["VSCODE" in x for x in os.environ.keys()]):
    print("Please enter password in the VS Code prompt at the top of your VS Code window!")
  os.environ["OPEN_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


### Set up config and environment variables
* NOTE: set the entity to your username or team name
* Set WandB environment variables to change behavior of logging
* `ENTITY` - username or team where your projects live
* `PROJECT` - project where your runs will live
* `LANGCHAIN_WANDB_TRACING` - automatically logs langchain traces, inputs and outputs as part of runs in WandB.

In [ ]:
from dataclasses import dataclass
from pathlib import Path
import os

project_name = "RAG-QA-Bot" #@param
entity = "matin-kh" #@param
TOTAL_EPISODES = 5

playlist_url = "https://www.youtube.com/playlist?list=PLD80i8An1OEEb1jP0sjEyiLG8ULRXFob_"
root_data_dir = Path("/contents/data")
root_artifact_dir = Path("downloaded_artifacts")
yt_podcast_data_artifact = f"{entity}/{project_name}/yt_podcast_transcript:latest"
summarized_data_artifact = f"{entity}/{project_name}/summarized_podcasts:latest"
summarized_que_data_artifact = f"{entity}/{project_name}/summarized_que_podcasts:latest"
transcript_embeddings_artifact = f"{entity}/{project_name}/transcript_embeddings:latest"

os.makedirs("/contents/data", exist_ok=True)
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ['WANDB_PROJECT'] = project_name
os.environ['WANDB_ENTITY'] = entity


### Log into WandB
You can explicitly login using `wandb login` or `wandb.login()` (see below)

Alternatively, you can set environment variables. There are several env variables which you can set to change the behavior of WandB logging. The most important ones are:
* `WANDB_API_KEY`: available in "Settings" section under your WandB profile.
* `WANDB_BASE_URL`: this is the url of the WandB server (if you are using a private instance)


In [ ]:
!wandb offline
!export CUDA_VISIBLE_DEVICES=-1

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [ ]:
!export WANDB_INIT_TIMEOUT=120
!export WANDB_HTTP_TIMEOUT=60

In [ ]:
!wandb online

W&B online. Running your script from this directory will now sync to the cloud.


In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import time
import pandas as pd
import wandb
from langchain.document_loaders import YoutubeLoader
from pytube import Playlist, YouTube
from tqdm import tqdm


def retry_access_yt_object(url, max_retries=5, interval_secs=5):
    """
    Retries creating a YouTube object with the given URL and accessing its title several times
    with a given interval in seconds, until it succeeds or the maximum number of attempts is reached.
    If the object still cannot be created or the title cannot be accessed after the maximum number
    of attempts, the last exception is raised.
    """
    last_exception = None
    for i in range(max_retries):
        try:
            yt = YouTube(url)
            title = yt.title  # Access the title of the YouTube object.
            return yt  # Return the YouTube object if successful.
        except Exception as err:
            last_exception = err  # Keep track of the last exception raised.
            print(
                f"Failed to create YouTube object or access title. Retrying... ({i+1}/{max_retries})"
            )
            time.sleep(interval_secs)  # Wait for the specified interval before retrying.

    # If the YouTube object still cannot be created or the title cannot be accessed after the maximum number of attempts, raise the last exception.
    raise last_exception

## 2. Process Data (Artifacts)

### Log Data Snapshots as Artifacts
WandB is unopiniated with regard to how you track your experiments. We could log data in any number of ways.
* Log one artifact which represents all the data - training, validation, and test
* Log several artifacts - one for each of the training, validation, and test data

### Anatomy of an artifact
The `Artifact` class will correspond to an entry in the WandB Artifact registry. The artifact has
* a name
* a type
* metadata
* description
* files, directory of files, or references

Example usage
```python
run wandb.init(project = "my-project")
artifact = wandb.Artifact(name="my_artifact", type="data")
artifact.add_file("/PATH/TO/MY/FILE.txt")
run.log_artifact(artifact)
run.finish
```

In [ ]:
run = wandb.init(project=project_name, entity=entity, job_type="dataset",
                 settings=wandb.Settings(start_method="fork"))

playlist = Playlist(playlist_url)
playlist_video_urls = playlist.video_urls[0:TOTAL_EPISODES]

print(f"There are total {len(playlist_video_urls)} videos in the playlist.")

video_data = []
for video in tqdm(playlist_video_urls, total=len(playlist_video_urls)):
  try:
    curr_video_data = {}
    yt = retry_access_yt_object(video, max_retries=25, interval_secs=2)
    curr_video_data["title"] = yt.title
    curr_video_data["url"] = video
    curr_video_data["duration"] = yt.length
    curr_video_data["publish_date"] = yt.publish_date.strftime("%Y-%m-%d")

    loader = YoutubeLoader.from_youtube_url(video)
    transcript = loader.load()[0].page_content
    transcript = " ".join(transcript.split())
    curr_video_data["transcript"] = transcript
    curr_video_data["total_words"] = len(transcript.split())

    video_data.append(curr_video_data)
  except Exception as inst:
    print(type(inst))
    print(inst.args)
    print(inst)
    print(f"Failed to scrape {video}")

print(f"Total podcast episodes scraped: {len(video_data)}")

# save the scraped data to a csv file
df = pd.DataFrame(video_data)
data_path = root_data_dir / "yt_podcast_transcript.csv"
df.to_csv(data_path, index=False)

# upload the scraped data to wandb
artifact = wandb.Artifact("yt_podcast_transcript", type="dataset")
artifact.add_file(data_path)
run.log_artifact(artifact)

wandb: Currently logged in as: matin-kh. Use `wandb login --relogin` to force relogin


There are total 5 videos in the playlist.


100%|██████████| 5/5 [00:07<00:00,  1.59s/it]


Total podcast episodes scraped: 5


<Artifact yt_podcast_transcript>

### Log (save) a WandB Table to interact with your data
* Log the dataframe of metadata about the youtube transcripts (urls, lengths, transcripts)
* This allows us to explore the original data (filtering, grouping, etc)

In [ ]:
# create WandB table
table = wandb.Table(dataframe=df)
run.log({"yt_podcast_transcript": table})
run.finish()

### Summarize YouTube Transcripts
Here we summarize the transcripts in chunks, summarizing each chunk and then summarizing the summaries using the LangChain `load_summarize_chain`.

We can do this in parallel since each chunk of a transcript can be summarized independently so we employ `map_reduce`.

In [ ]:
import os
import pandas as pd
from langchain.callbacks import get_openai_callback
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import DataFrameLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from tqdm import tqdm
import wandb

def get_data(artifact_name: str, total_episodes: int=None):
  podcast_artifact = wandb.use_artifact(artifact_name)
  podcast_artifact_dir = podcast_artifact.download(root_artifact_dir)
  filename = artifact_name.split(":")[0].split("/")[-1]
  df = pd.read_csv(os.path.join(podcast_artifact_dir, f"{filename}.csv"))
  if total_episodes is not None:
    df = df.iloc[:total_episodes]

  return df


def summarize_episodes(episode_df: pd.DataFrame):
  # load docs into langchain format
  loader = DataFrameLoader(episode_df, page_content_column="transcript")
  data = loader.load()

  # split the documents
  text_splitter = TokenTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=0)
  docs = text_splitter.split_documents(data)
  print(f"Number of documents for podcast {data[0].metadata['title']}: {len(docs)}")

  # Initialize LLM
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

  # define map prompt
  map_prompt = """Write a concise summary of the following short transcript from a podcast.
  Don't add your opinions or interpretations.

  {text}

  CONCISE SUMMARY:"""

  # define combine prompt
  combine_prompt = """You have been provided with summaries of chunks of transcripts from a podcast.
  Your task is to merge these intermediate summaries to create a brief and comprehensive summary of the entire podcast.
  The summary should encompass all the crucial points of the podcast.
  Ensure that the summary is atleast 2 paragraph long and effectively captures the essence of the podcast.
  {text}

  SUMMARY:"""

  map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])
  combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

  # initialize the summarizer chain
  chain = load_summarize_chain(
      llm,
      chain_type="map_reduce",
      return_intermediate_steps=True,
      map_prompt=map_prompt_template,
      combine_prompt=combine_prompt_template,
  )

  summary = chain({"input_documents": docs})

  return summary

### Execute Summary chain and log (save) results
* You can instantiate a `WandBTracer` and pass in additional config about this LangChain run.
* Log the outputs of the chain like tokens used, cost, etc
* Log the resulting summaries as artifacts.

In [ ]:
from langchain.callbacks.tracers import WandbTracer

tracer = WandbTracer(run_args = {"job_type": "summarize"})

# get scraped data
df = get_data(artifact_name=yt_podcast_data_artifact, total_episodes=TOTAL_EPISODES)

summaries = []
with get_openai_callback() as cb:
  for episode in tqdm(df.iterrows(), total=len(df), desc="Summarizing episodes"):
    episode_data = episode[1].to_frame().T

    summary = summarize_episodes(episode_data)
    summaries.append(summary["output_text"])

  print("*" * 25)
  print(cb)
  print("*" * 25)

  wandb.log(
      {
          "total_prompt_tokens": cb.prompt_tokens,
          "total_completion_tokens": cb.completion_tokens,
          "total_tokens": cb.total_tokens,
          "total_cost": cb.total_cost,
      }
  )

df["summary"] = summaries

# save data
path_to_save = os.path.join(root_data_dir, "summarized_podcasts.csv")
df.to_csv(path_to_save, index=False)

# log to WandB artifact
artifact = wandb.Artifact("summarized_podcasts", type="dataset")
artifact.add_file(path_to_save)
wandb.log_artifact(artifact)

# create wandb table
table = wandb.Table(dataframe=df)
wandb.log({"summarized_podcasts": table})

tracer.finish()

wandb: Streaming LangChain activity to W&B at https://wandb.ai/matin-kh/RAG-QA-Bot/runs/fegegzc6
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.
wandb:   1 of 1 files downloaded.  
Summarizing episodes:   0%|          | 0/5 [00:00<?, ?it/s]

Number of documents for podcast Transforming Data into Business Solutions with Salesforce AI CEO, Clara Shih: 12


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
wandb: WARNING WARNING: Failed to serialize model: 'int' object is not subscriptable
Summarizing episodes:   0%|          | 0/5 [00:06<?, ?it/s]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Embed the contents of YouTube Transcripts
Here we use OpenAI embeddings and ChromaDB to embed the summaries to make them queriable via vector similarity search when we ask contextual questions to the LLM.

We use `wandb.log` to save the resulting ChromaDB serialized embeddings as artifacts.

In [ ]:
# Setup ChromaDB

import os
from dataclasses import asdict

import pandas as pd
from langchain.callbacks import get_openai_callback
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Chroma
from tqdm import tqdm
from wandb.integration.langchain import WandbTracer

import wandb


def get_data(artifact_name: str, total_episodes=None):
    podcast_artifact = wandb.use_artifact(artifact_name, type="dataset")
    podcast_artifact_dir = podcast_artifact.download(root_artifact_dir)
    filename = artifact_name.split(":")[0].split("/")[-1]
    df = pd.read_csv(os.path.join(podcast_artifact_dir, f"{filename}.csv"))
    if total_episodes is not None:
        df = df.iloc[:total_episodes]
    return df


def create_embeddings(episode_df: pd.DataFrame, index: int):
    # load docs into langchain format
    loader = DataFrameLoader(episode_df, page_content_column="transcript")
    data = loader.load()

    # split the documents
    text_splitter = TokenTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(data)

    title = data[0].metadata["title"]
    print(f"Number of documents for podcast {title}: {len(docs)}")

    # initialize embedding engine
    embeddings = OpenAIEmbeddings()

    db = Chroma.from_documents(
        docs,
        embeddings,
        persist_directory=os.path.join(root_data_dir / "chromadb", str(index)),
    )
    db.persist()

In [ ]:
# Embedding part

tracer = WandbTracer(run_args = {"job_type": "embed_transcripts"})

# get data
df = get_data(artifact_name=summarized_data_artifact, total_episodes=TOTAL_EPISODES)

# create embeddings
with get_openai_callback() as cb:
    for episode in tqdm(df.iterrows(), total=len(df), desc="Embedding transcripts"):
        episode_data = episode[1].to_frame().T

        create_embeddings(episode_data, index=episode[0])

    print("*" * 25)
    print(cb)
    print("*" * 25)

    wandb.log(
        {
            "total_prompt_tokens": cb.prompt_tokens,
            "total_completion_tokens": cb.completion_tokens,
            "total_tokens": cb.total_tokens,
            "total_cost": cb.total_cost,
        }
    )

# log embeddings to wandb artifact
artifact = wandb.Artifact("transcript_embeddings", type="dataset")
artifact.add_dir(root_data_dir / "chromadb")
wandb.log_artifact(artifact)

tracer.finish()

wandb: WARNING This feature is deprecated and has been moved to `langchain`. Enable tracing by setting LANGCHAIN_WANDB_TRACING=true in your environment. See the documentation at https://python.langchain.com/docs/ecosystem/integrations/agent_with_wandb_tracing for guidance. Replace your current import with `from langchain.callbacks.tracers import WandbTracer`.


CommError: Unable to fetch artifact with name matin-kh/RAG-QA-Bot/summarized_podcasts:latest

## Ask Questions Against the Summarized Documents
Finally we tie everything together.
1. We can pull down our ChromaDB embeddings from W&B
2. Pass them along with a prompt template for QA to the `RetrievalQA` chain and start asking questions!

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings

def get_answer(podcast: str, question: str):
  index = df[df["title"] == podcast].index[0]
  db_dir = os.path.join(chromadb_dir, str(index))
  embeddings = OpenAIEmbeddings()
  db = Chroma(persist_directory=db_dir, embedding_function=embeddings)

  prompt_template = """Use the following pieces of context to answer the question.
  If you don't know the answer, just say that you don't know, don't try to make up an answer.
  Don't add your opinions or interpretations. Ensure that you complete the answer.
  If the question is not relevant to the context, just say that it is not relevant.

  CONTEXT:
  {context}

  QUESTION: {question}

  ANSWER:"""

  prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

  retriever = db.as_retriever()
  retriever.search_kwargs["k"] = 2

  qa = RetrievalQA.from_chain_type(
      llm=ChatOpenAI(temperature=0),
      chain_type="stuff",
      retriever=retriever,
      chain_type_kwargs={"prompt": prompt},
      return_source_documents=True,
  )

  with get_openai_callback() as cb:
      result = qa({"query": question})
      print(cb)

  answer = result["result"]
  return answer

In [ ]:
# download and read data
api = wandb.Api()
artifact_df = api.artifact(summarized_data_artifact)
artifact_df.download(root_data_dir)

artifact_embeddings = api.artifact(transcript_embeddings_artifact)
chromadb_dir = artifact_embeddings.download(root_data_dir / "chromadb")

df_path = root_data_dir / "summarized_podcasts.csv"
df = pd.read_csv(df_path)

CommError: Unable to fetch artifact with name matin-kh/RAG-QA-Bot/summarized_podcasts:latest

In [ ]:
df["title"].tolist()[0:TOTAL_EPISODES]

['Transforming Data into Business Solutions with Salesforce AI CEO, Clara Shih',
 'Upgrading Your Health: Navigating AIs Future In Healthcare with John Halamka of Mayo Clinic Platform',
 'Shaping the World of Robotics with Chelsea Finn',
 "The Power of AI in Search with You.com's Richard Socher",
 "AI's Future: Investment & Impact with Sarah Guo and Elad Gil of the No Priors Podcast"]

In [ ]:
tracer = WandbTracer(run_args = {"job_type": "retriealQA"})

answer = get_answer("The Power of AI in Search with You.com's Richard Socher", "Who won the 2024 superbowl?")
print(answer)

tracer.finish()

NameError: name 'chromadb_dir' is not defined

## Build an App to Serve the Model

In [ ]:
import os
import re
from ast import literal_eval

import wandb
import gradio as gr
import pandas as pd
from langchain.callbacks import get_openai_callback
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma

from dataclasses import dataclass
from pathlib import Path


@dataclass
class Config:
    playlist_url: str = "https://www.youtube.com/playlist?list=PLD80i8An1OEEb1jP0sjEyiLG8ULRXFob_"

    # paths
    root_data_dir: Path = Path("data")
    root_artifact_dir: Path = Path("downloaded_artifacts")

    # wandb
    project_name: str = "gradient_dissent_qabot"
    yt_podcast_data_artifact: str = "gladiator/gradient_dissent_qabot/yt_podcast_transcript:latest"
    summarized_data_artifact: str = "gladiator/gradient_dissent_qabot/summarized_podcasts:latest"
    summarized_que_data_artifact: str = (
        "gladiator/gradient_dissent_qabot/summarized_que_podcasts:latest"
    )
    transcript_embeddings_artifact: str = (
        "gladiator/gradient_dissent_qabot/transcript_embeddings:latest"
    )

config = Config()

# download and read data
api = wandb.Api()
artifact_df = api.artifact(config.summarized_que_data_artifact)
artifact_df.download(config.root_data_dir)

artifact_embeddings = api.artifact(config.transcript_embeddings_artifact)
chromadb_dir = artifact_embeddings.download(config.root_data_dir / "chromadb")

df_path = config.root_data_dir / "summarized_que_podcasts.csv"
df = pd.read_csv(df_path)


def embed_video(title: str):
    video_url = df[df["title"] == title]["url"].values[0]
    match = re.search(r"v=([-\w]+)", video_url)
    video_id = match.group(1)
    # embed video
    video_embed = f"<iframe width='580' height='360' src=https://www.youtube.com/embed/{video_id} frameborder='0' allowfullscreen style='width:100%; max-width:100%;'></iframe>"

    return video_embed


def get_podcast_info(title: str):
    # get questions
    questions = df[df["title"] == title]["questions"].values[0]
    questions = literal_eval(questions)
    que_str = ""
    for que in questions:
        que_str += f"👉 {que}\n"

    # get summary
    summary = df[df["title"] == title]["summary"].values[0]

    return summary, que_str


def get_answer(podcast: str, question: str):
    index = df[df["title"] == podcast].index[0]
    db_dir = os.path.join(chromadb_dir, str(index))
    embeddings = OpenAIEmbeddings()
    db = Chroma(persist_directory=db_dir, embedding_function=embeddings)

    prompt_template = """Use the following pieces of context to answer the question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Don't add your opinions or interpretations. Ensure that you complete the answer.
    If the question is not relevant to the context, just say that it is not relevant.

    CONTEXT:
    {context}

    QUESTION: {question}

    ANSWER:"""

    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    retriever = db.as_retriever()
    retriever.search_kwargs["k"] = 2

    qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(temperature=0),
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True,
    )

    with get_openai_callback() as cb:
        result = qa({"query": question})
        print(cb)

    answer = result["result"]
    return answer


with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Welcome to Gradient Dissent QA Bot 🤖</center></h1>")
    gr.Markdown(
        "#### The purpose of this QA bot is to provide answers to questions related to podcast episodes from Weights & Biases' [Gradient Dissent Podcast](https://www.youtube.com/playlist?list=PLD80i8An1OEEb1jP0sjEyiLG8ULRXFob_)."
    )
    gr.Markdown(
        "#### First select a podcast episode and click `Get Podcast Info` to get the summary and possible questions about the podcast episode."
    )
    gr.Markdown(
        "#### Then ask a question about the podcast episode and click `Get Answer` to get the answer."
    )
    gr.Markdown(
        "#### Read the report for understanding how I built this QA bot [here](https://wandb.ai/gladiator/gradient_dissent_qabot/reports/Building-a-Q-A-Bot-for-Weights-Biases-Gradient-Dissent-Podcast--Vmlldzo0MTcyMDQz)"
    )
    gr.Markdown(
        "#### GitHub Repo [here](https://github.com/Gladiator07/wandb-gradient-dissent-bot/tree/main)"
    )
    gr.Markdown("<br>")

    with gr.Row():
        with gr.Column(scale=0.5):
            dropdown = gr.Dropdown(
                df["title"].to_list(), label="Select a Podcast Episode", value=df.iloc[0]["title"]
            )
            podcast_info_btn = gr.Button("Get Podcast Info")

            podcast_info_btn.click(
                fn=embed_video,
                inputs=dropdown,
                outputs=gr.HTML(label="Podcast Video"),
            )

            question_box = gr.Textbox(label="Ask a question about the podcast episode")
            with gr.Row():
                ques_clear_btn = gr.Button("Clear")
                ques_btn = gr.Button("Get Answer")

            ques_btn.click(
                fn=get_answer,
                inputs=[dropdown, question_box],
                outputs=gr.Textbox(label="Answer"),
            )
            ques_clear_btn.click(lambda: None, None, question_box, queue=False)

        with gr.Column(scale=0.5):
            podcast_info_btn.click(
                fn=get_podcast_info,
                inputs=dropdown,
                outputs=[
                    gr.Text(label="Summary of the podcast"),
                    gr.Text(label="Some of the questions you can ask"),
                ],
            )


demo.launch()

wandb:   1 of 1 files downloaded.  
wandb:   492 of 492 files downloaded.  
/usr/local/lib/python3.10/dist-packages/gradio/layouts/column.py:53: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://905992333697cab128.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
